In [1]:
import timeit
from datetime import datetime
import requests
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
import urllib
import json

In [2]:
if "auth_cookie" not in locals():
    auth_cookie = None

autorun_execute_count = 0
columns = ['EPOCH', 'NORAD_CAT_ID', 'MEAN_MOTION', 'ECCENTRICITY', 'INCLINATION', 'RA_OF_ASC_NODE', 'ARG_OF_PERICENTER', 'MEAN_ANOMALY', 'REV_AT_EPOCH', 'BSTAR', 'MEAN_MOTION_DOT', 'MEAN_MOTION_DDOT', 'SEMIMAJOR_AXIS', 'PERIOD', 'APOAPSIS', 'PERIAPSIS']

In [3]:
try:
    logs = pd.read_pickle("./logs.pkl.gz","gzip")
except:
    # time, type (trigger, login, skip, fetch_init, fetch_success, process_complete, error), url, epoch, out file
    logs = pd.DataFrame(columns = ["created_on","type","url","epoch","error","output"])
    logs = logs.astype({'created_on': 'datetime64[ns]'})
    
def save_logs():
    logs.to_pickle("./logs.pkl.gz","gzip")
    
def log(log_type,url=None,epoch=None,error=None,output=None):
    global logs
    new_log = {'created_on':np.datetime64('now'), 'type':log_type, 'url':url, 'epoch':epoch, 'error':error, 'output':output}
    logs = logs.append(new_log,ignore_index=True)
    save_logs()

In [4]:
def get_current_file_count():
    return max([int(f[:-7]) for f in listdir('./output/') if f.endswith(".pkl.gz")])

def get_last_epoch():
    df = pd.read_pickle("./output/%05d.pkl.gz" % get_current_file_count(),"gzip")
    return df.EPOCH[-1:].astype("datetime64[ns]").values[0]

def get_space_track_api_count(unit="m"):
    return len(logs[(logs.created_on > (np.datetime64('now') - np.timedelta64(1,unit))) & (logs.type=="fetch_init")])

def space_track_login():
    with open("space-track-credentials.json") as json_file:
        credentials = json.load(json_file)
    print("Logging in space-track.org with identity:",credentials['identity'])
    url = "https://www.space-track.org/ajaxauth/login"
    x = requests.post(url, data = credentials)
    if x.status_code == 200:
        print("Logged in")
        log("login", output="success")
        return x.cookies
    else:
        log("login", error=x.status_code)
        print("Failed with code:",x.status_code)
        return None
    
def generate_url():
    lower = get_last_epoch()
    l = np.datetime_as_string(lower, unit='us')
    if lower.astype('datetime64[Y]').item().year > 2014:
        upper = lower + np.timedelta64(60,'D') # after 2014
    else:
        upper = lower + np.timedelta64(365,'D') # before 2014
    u = np.datetime_as_string(upper, unit='us')
    return (l, u, f'https://www.space-track.org/basicspacedata/query/class/gp_history/ECCENTRICITY/<0.25/MEAN_MOTION//>11.25/EPOCH/>{l},<{u}/OBJECT_TYPE/PAYLOAD/orderby/EPOCH asc/limit/100000/format/csv/emptyresult/show')

In [5]:
# https://www.space-track.org/basicspacedata/query/class/gp_history/ECCENTRICITY/<0.25/MEAN_MOTION//>11.25/EPOCH/>1957-01-01,<1970-01-01/OBJECT_TYPE/PAYLOAD/orderby/EPOCH asc/limit/100000/format/csv/emptyresult/show

Before you proceed:
* Create the `space-track-credentials.json` file from template with your own credentials
* Manually test the fetch cell to make sure things are working first
* Set an appropriate intervalTime
* This will probably mess up if your task time is longer than your interval time
* If you don't know what you are doing, ask Tim.

In [6]:
raise Exception('This exception is to stop the cells below from running when you do "Run All Cells"')

Exception: This exception is to stop the cells below from running when you do "Run All Cells"

In [ ]:
%%html
<script>
    var intervalTime = 300000; // 5 minutes in ms
    if (typeof autorun_toggle === 'undefined') {
        var autorun_toggle = null;
    }
    function toggle_autorun() {
        var btn = document.getElementById("autorun_button");
        setTimeout(function(){ // add a delay so the selected cell is correctly set here
            if (autorun_toggle == null) {
                var start_index = IPython.notebook.get_selected_index()+1;
                var end_index = IPython.notebook.get_cells().length;
                console.log("start autorun, start: " + start_index + " end: " + end_index);
                IPython.notebook.execute_cell_range(start_index, end_index);
                //$('div.input').hide(200);
                autorun_toggle = setInterval(function(){
                    console.log("Run cells below");
                    //IPython.notebook.execute_cells_below();
                    IPython.notebook.execute_cell_range(start_index, end_index);
                }, intervalTime);
            } else {
                window.clearInterval(autorun_toggle);
                console.log("clear autorun");
                autorun_toggle = null;
                //$('div.input').show(200);
            }
        }, 100);
    }
</script>
<button id="autorun_button" onclick="toggle_autorun()">Toggle autorun cells below</button>

In [ ]:
start_time = timeit.default_timer()
autorun_execute_count+=1
print(f'Last run local time: {datetime.now()}')
print(f'Total times ran: {autorun_execute_count}')

In [6]:
# Check that we did not exceed Space-Track API limits
# Limit API queries to less than 30 requests per minute / 300 requests per hour

ops_count = 0

while get_space_track_api_count("m") < 25 and get_space_track_api_count("h") < 275:
    ops_count += 1
    # clear expired cookies
    print("------")
    print(f"Local time: {datetime.now()}")
    if type(auth_cookie) == requests.cookies.RequestsCookieJar:
        auth_cookie.clear_expired_cookies()
    # log in if needed
    if auth_cookie == None or len(auth_cookie) == 0:
        auth_cookie = space_track_login()
    # stop if can't log in
    if auth_cookie == None:
        log("skip", error = f'Cannot log in')
        break
    # prep url
    start, _, url = generate_url()
    print(f"Fetch at epoch >: {start}")
    log("fetch_init", url=url, epoch=start)
    r = requests.get(url, allow_redirects=True, cookies=auth_cookie)
    fname = f"./tmp/{start}.csv".replace(":","_")
    open(fname, 'wb').write(r.content)
    print(f"Saved CSV: {fname}")
    log("fetch_success", url=url, epoch=start, output=fname)
    oname = ("./output/%05d.pkl.gz" % (get_current_file_count()+1)).replace(":","_")
    pd.read_csv(fname)[columns].to_pickle(oname,"gzip")
    log("process_complete", url=url, epoch=start, output=oname)
    print(f"Processed to: {oname}")
    
if ops_count == 0:
    log("skip", error = f'Request limit exceeded: 1m({get_space_track_api_count("m")}) 1h({get_space_track_api_count("h")})')
else:
    log("stop", output=ops_count, error = f'Request limit exceeded: 1m({get_space_track_api_count("m")}) 1h({get_space_track_api_count("h")})')

------
Local time: 2021-01-22 18:09:44.992058
Logging in space-track.org with identity: ttcchen@umich.edu
Logged in
Fetch at epoch >: 2021-01-14T07:37:28.927488
Saved CSV: ./tmp/2021-01-14T07_37_28.927488.csv
Processed to: ./output/00336.pkl.gz
------
Local time: 2021-01-22 18:10:16.798653
Fetch at epoch >: 2021-01-21T20:33:44.796672
Saved CSV: ./tmp/2021-01-21T20_33_44.796672.csv
Processed to: ./output/00337.pkl.gz
------
Local time: 2021-01-22 18:10:31.639855
Fetch at epoch >: 2021-01-22T06:00:00.999648
Saved CSV: ./tmp/2021-01-22T06_00_00.999648.csv
Processed to: ./output/00338.pkl.gz
------
Local time: 2021-01-22 18:10:37.435627
Fetch at epoch >: 2021-01-22T06:00:00.999648
Saved CSV: ./tmp/2021-01-22T06_00_00.999648.csv
Processed to: ./output/00339.pkl.gz
------
Local time: 2021-01-22 18:10:45.674503
Fetch at epoch >: 2021-01-22T06:00:00.999648


KeyboardInterrupt: 

In [ ]:
elapsed = timeit.default_timer() - start_time
print(f"Time used to execute cells: {elapsed} seconds")

In [ ]:
logs